In [ ]:
import numpy as np
import os
import cv2
from utils import *
%matplotlib notebook

# 1. Data Generation for YOLOv2

In [ ]:
#############################
### Simulation Parameters ###
#############################
x_c = np.array([0, 20, 20])
f = 10
sensor_w = 6
sensor_h = 4

u_scale = 100/180*np.pi
t_max = 10
fps = 10
x_max = 40

In [ ]:
def yolo_data_generation(f, ax2, vehicles, sensor_w, sensor_h, dpi=100, 
                         file_template="Virtual_FOV_frame_{0}_it_0.{1}", 
                         file_path=".\\yolov3\\object-detection\\raw_data"):
    # Get window extent
    extent = ax2.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    extent.x0 = extent.x0 + 0.02
    extent.x1 = extent.x1 - 0.02
    extent.y0 = extent.y0 + 0.02
    extent.y1 = extent.y1 - 0.02
    extent_delta_x = (extent.x1 - extent.x0) * dpi
    extent_delta_y = (extent.y1 - extent.y0) * dpi
    # Get file paths
    image_path = os.path.join(file_path, file_template.format(f, "jpg"))
    labels_path = os.path.join(file_path, file_template.format(f, "txt"))
    # Save image
    ax2.figure.savefig(image_path, dpi=dpi, bbox_inches=extent)
    # Get vehicle bounding boxes in yolo format
    f = open(labels_path, 'w')
    for vehicle in vehicles:
        vv_corners = vehicle.vv_corners.get_offsets()
        box_center_x = (np.min(vv_corners[:, 0])+np.max(vv_corners[:, 0]))/2
        box_center_y = (np.min(vv_corners[:, 1])+np.max(vv_corners[:, 1]))/2
        box_width_x = (np.max(vv_corners[:, 0])-np.min(vv_corners[:, 0]))*1.07 # Margin
        box_height_y = (np.max(vv_corners[:, 1])-np.min(vv_corners[:, 1]))*1.07 # Margin
        # Do something only if box in FOV
        if (box_center_x > -sensor_w/2) and (box_center_x <= sensor_w/2) and (box_center_y > -sensor_h/2) and (box_center_y < sensor_h/2):
            # box center is in FOV
            # Check if box is clipped somewhere
            if box_center_x + box_width_x/2 > sensor_w/2:
                delta_clip = (box_center_x + box_width_x/2) - sensor_w/2
                box_center_x -= delta_clip/2
                box_width_x -= delta_clip
            if box_center_x + box_width_x/2 < -sensor_w/2:
                delta_clip = - sensor_w/2 - (box_center_x + box_width_x/2)
                box_center_x += delta_clip/2
                box_width_x -= delta_clip
            if box_center_y + box_height_y/2 > sensor_h/2:
                delta_clip = (box_center_y + box_height_y/2) - sensor_h/2
                box_center_y -= delta_clip/2
                box_height_y -= delta_clip
            if box_center_y + box_height_y/2 < -sensor_h/2:
                delta_clip = - sensor_h/2 - (box_center_y + box_height_y/2)
                box_center_y += delta_clip/2
                box_height_y -= delta_clip
            # Now convert to image coordinates
            box_center_x_img = box_center_x / sensor_w + 1/2
            box_center_y_img = 1 - (box_center_y / sensor_h + 1/2)
            box_width_x_img = box_width_x/sensor_w
            box_height_y_img = box_height_y/sensor_h
            f.write(f"0 {box_center_x_img} {box_center_y_img} {box_width_x_img} {box_height_y_img}\n")
    f.close()

In [ ]:
def update(f, suffix=""):
    if f == 0:
        phi_t = phi_0
    t = f/fps
    if f >= t_max*fps:
        return
    psi_t = psi_0 + np.sum(phi_dots[:f, 0])/fps
    phi_t = phi_0 + np.sum(phi_dots[:f, 1])/fps
    # Update objects
    camera_fov.update_FOV(psi=psi_t, phi=phi_t)
    for v in vehicles:
        v.update_vehicle(t)
        v.update_vehicle_virtual(camera_fov, t)
    for b in background:
        b.update_vehicle_virtual(camera_fov)
    # Update angle plot
    psi_data[1, f] = psi_t
    psi_line.set_data(psi_data[:, :(f+1)])
    phi_data[1, f] = phi_t
    phi_line.set_data(phi_data[:, :(f+1)])
    # Update u_plot
    psi_dot, phi_dot = psi_phi_dot(t, t_max)
    u1_data[1, f] = psi_dot/u_scale
    u1_line.set_data(u1_data[:, :(f+1)])
    u2_data[1, f] = phi_dot/u_scale
    u2_line.set_data(u2_data[:, :(f+1)])
    if (f+1)%fps == 0:
        title.set_text(
            'PT Camera Simulation, time={t:0.0f}s\nApplication: {app_name}'.format(t=t, app_name=app_name))
    file_path = ".\\yolov3\\object-detection\\raw_data"
    if not os.path.exists(file_path):
        os.makedirs(file_path)
    # Save virtual FOV figures
    yolo_data_generation(f, ax2, vehicles+background, sensor_w, sensor_h, dpi=100, 
                         file_template="Virtual_FOV_frame_{0}_"+f"{suffix}"+".{1}", 
                         file_path=".\\yolov3\\object-detection\\raw_data")

In [ ]:
sim_configs = [
    [
        [
            lambda ax, ax2: LinearVehicle(ax, ax2, v_l=2, v_w=4, vel_dir=np.array([2, 1.4])*4, color='gray'),
            lambda ax, ax2: LinearVehicle(ax, ax2, v_l=2, v_w=4, y_offset=40, vel_dir=np.array([1, -1])*2, color='burlywood'),
            lambda ax, ax2: LinearVehicle(ax, ax2, v_l=2, v_w=4, x_offset=20, y_offset=40,  vel_dir=np.array([1, -1])*2, color='black'),
            lambda ax, ax2: LinearVehicle(ax, ax2, v_l=2, v_w=4, x_offset=25, y_offset=0,  vel_dir=np.array([-1, 4]), color='saddlebrown'),
        ],
        [
            lambda ax, ax2: BackgroundObject(ax, ax2, v_l=4, v_w=4, x_offset=10, y_offset=10, vel_dir=np.array([0, 0]), color="darkgreen"),
            lambda ax, ax2: BackgroundObject(ax, ax2, v_l=3, v_w=4, x_offset=10, y_offset=30, vel_dir=np.array([0, 0]), color="slategray"),
            lambda ax, ax2: BackgroundObject(ax, ax2, v_l=3, v_w=4, x_offset=30, y_offset=10,  vel_dir=np.array([0, 1]), color="sienna"),
            lambda ax, ax2: BackgroundObject(ax, ax2, v_l=6, v_w=4, x_offset=25, y_offset=25,  vel_dir=np.array([0, 0]), color="steelblue"),
        ],
        [
            {"suffix":"conf_0_it_0", "psi_0": 0, "phi_0": np.pi-np.pi/5, 
             "psi_phi_dot": lambda t, t_max: (2*np.pi/(t_max), 0) if t <= t_max/2 else (-2*np.pi/(t_max-1/2), 0)},
            {"suffix":"conf_0_it_1", "psi_0": np.pi/3, "phi_0": np.pi-np.pi/3.6, 
             "psi_phi_dot": lambda t, t_max: (2*np.pi/3/(t_max), 0) if t <= t_max/2 else (-2*np.pi/3/(t_max-1/2), 0)},
            {"suffix":"conf_0_it_2", "psi_0": np.pi/2, "phi_0": np.pi-np.pi/15, 
             "psi_phi_dot": lambda t, t_max: (0, -2*np.pi/4/(t_max)) if t <= t_max/2 else (0, 2*np.pi/4/(t_max-1/2))}
        ]
    ],
    [
        [
            lambda ax, ax2: LinearVehicle(ax, ax2, v_l=2, v_w=4, x_offset=5, y_offset=20, vel_dir=np.array([1, 0.1])*4, color='gray'),
            lambda ax, ax2: LinearVehicle(ax, ax2, v_l=2, v_w=4, x_offset=20, y_offset=0, vel_dir=np.array([1, 4])*1.5, color='burlywood'),
            lambda ax, ax2: LinearVehicle(ax, ax2, v_l=2, v_w=4, x_offset=40, y_offset=10,  vel_dir=np.array([-1.2, 1])*2, color='black'),
            lambda ax, ax2: LinearVehicle(ax, ax2, v_l=2, v_w=4, x_offset=25, y_offset=0,  vel_dir=np.array([-1, 4]), color='saddlebrown'),
        ],
        [
            lambda ax, ax2: BackgroundObject(ax, ax2, v_l=2, v_w=4, x_offset=20, y_offset=30, vel_dir=np.array([0, 0]), color="darkgreen"),
            lambda ax, ax2: BackgroundObject(ax, ax2, v_l=5, v_w=2, x_offset=30, y_offset=5, vel_dir=np.array([0, 0]), color="slategray"),
            lambda ax, ax2: BackgroundObject(ax, ax2, v_l=3, v_w=4, x_offset=30, y_offset=25,  vel_dir=np.array([0.5, 1]), color="sienna"),
            lambda ax, ax2: BackgroundObject(ax, ax2, v_l=5, v_w=5, x_offset=10, y_offset=15,  vel_dir=np.array([1, 1]), color="steelblue"),
        ],
        [
            {"suffix":"conf_1_it_0", "psi_0": 0, "phi_0": np.pi-np.pi/5, 
             "psi_phi_dot": lambda t, t_max: (2*np.pi/(t_max), 0) if t <= t_max/2 else (-2*np.pi/(t_max-1/2), 0)},
            {"suffix":"conf_1_it_1", "psi_0": np.pi/3, "phi_0": np.pi-np.pi/3.6, 
             "psi_phi_dot": lambda t, t_max: (2*np.pi/3/(t_max), 0) if t <= t_max/2 else (-2*np.pi/3/(t_max-1/2), 0)},
            {"suffix":"conf_1_it_2", "psi_0": np.pi/2, "phi_0": np.pi-np.pi/15, 
             "psi_phi_dot": lambda t, t_max: (0, -2*np.pi/4/(t_max)) if t <= t_max/2 else (0, 2*np.pi/4/(t_max-1/2))}
        ]
    ],
    [
        [
            lambda ax, ax2: LinearVehicle(ax, ax2, v_l=2, v_w=4, x_offset=0, y_offset=30, vel_dir=np.array([1, -1.1])*2, color='gray'),
            lambda ax, ax2: LinearVehicle(ax, ax2, v_l=2, v_w=4, x_offset=20, y_offset=0, vel_dir=np.array([1, 4])*1.5, color='burlywood'),
            lambda ax, ax2: LinearVehicle(ax, ax2, v_l=2, v_w=4, x_offset=40, y_offset=10,  vel_dir=np.array([-1.2, 1])*3, color='black'),
            lambda ax, ax2: LinearVehicle(ax, ax2, v_l=2, v_w=4, x_offset=15, y_offset=10,  vel_dir=np.array([1, 1])*2, color='saddlebrown'),
        ],
        [
            lambda ax, ax2: BackgroundObject(ax, ax2, v_l=4, v_w=4, x_offset=5, y_offset=30, vel_dir=np.array([1, 1]), color="darkgreen"),
            lambda ax, ax2: BackgroundObject(ax, ax2, v_l=5, v_w=2, x_offset=10, y_offset=10, vel_dir=np.array([1, 0.1]), color="slategray"),
            lambda ax, ax2: BackgroundObject(ax, ax2, v_l=2, v_w=4, x_offset=30, y_offset=10,  vel_dir=np.array([0, 1]), color="sienna"),
            lambda ax, ax2: BackgroundObject(ax, ax2, v_l=3, v_w=6, x_offset=15, y_offset=20,  vel_dir=np.array([1, 0]), color="steelblue"),
        ],
        [
            {"suffix":"conf_2_it_0", "psi_0": 0, "phi_0": np.pi-np.pi/5, 
             "psi_phi_dot": lambda t, t_max: (2*np.pi/(t_max), 0) if t <= t_max/2 else (-2*np.pi/(t_max-1/2), 0)},
            {"suffix":"conf_2_it_1", "psi_0": np.pi/3, "phi_0": np.pi-np.pi/3.6, 
             "psi_phi_dot": lambda t, t_max: (2*np.pi/3/(t_max), 0) if t <= t_max/2 else (-2*np.pi/3/(t_max-1/2), 0)},
            {"suffix":"conf_2_it_2", "psi_0": np.pi/2, "phi_0": np.pi-np.pi/15, 
             "psi_phi_dot": lambda t, t_max: (0, -2*np.pi/4/(t_max)) if t <= t_max/2 else (0, 2*np.pi/4/(t_max-1/2))}
        ]
    ]
]

In [ ]:
###########################
### Wildlife Simulation ###
###########################

np.random.seed(5)
app_name = "Wildlife tracking"

config_id = 2
it_id = 2

conf_dict = sim_configs[config_id][2][it_id]

psi_0 = conf_dict["psi_0"]
phi_0 = conf_dict["phi_0"]

fig, ax, ax2, camera_fov, psi_line, psi_data, phi_line, \
phi_data, u1_line, u1_data, u2_line, u2_data = animation_setup(sensor_w, sensor_h, f, x_c, psi_0, phi_0, t_max, fps)

vehicles = [vehicle_f(ax,ax2) for vehicle_f in sim_configs[config_id][0]]
background = [background_f(ax,ax2) for background_f in sim_configs[config_id][1]]

psi_phi_dot = conf_dict["psi_phi_dot"]

phi_dots = np.array([psi_phi_dot(f/fps, t_max) for f in np.arange(fps*t_max)])
title = ax.set_title('PT Camera Simulation, time=0s\nApplication: {}'.format(app_name))

# Animate FOV
ani = animation.FuncAnimation(fig, update, t_max*fps+10, fargs=(conf_dict["suffix"],), interval=10, blit=True, repeat=False)
#ani.save('PT_camera_simulation_wildlife_16_9.gif', fps=fps)#, writer='imagemagick')
#plt.savefig("PT_camera_simulation_wildlife_snapshot2.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
# Cleanup
non_empty = 0
suffixes = ["conf_0_it_0", "conf_0_it_1", "conf_0_it_2", 
            "conf_1_it_0", "conf_1_it_1", "conf_1_it_2", 
            "conf_2_it_0", "conf_2_it_1", "conf_2_it_2"]
for suffix in suffixes:
    for ff in range(100):
        bboxes = np.loadtxt(f'.\\yolov3\\object-detection\\raw_data\\Virtual_FOV_frame_{ff}_{suffix}.txt')
        if bboxes.size!=0:
            non_empty += 1
        if bboxes.size==0:
            os.remove(f'.\\yolov3\\object-detection\\raw_data\\Virtual_FOV_frame_{ff}_{suffix}.txt')
            os.remove(f'.\\yolov3\\object-detection\\raw_data\\Virtual_FOV_frame_{ff}_{suffix}.jpg')

In [ ]:
# Create train and test metadata
train_file = f".\\yolov3\\object-detection\\train.txt"
test_file = f".\\yolov3\\object-detection\\test.txt"
path_template = "/content/object-detection/raw_data/{0}.jpg"
suffixes = ["conf_0_it_0", "conf_0_it_1", "conf_0_it_2", 
            "conf_1_it_0", "conf_1_it_1", "conf_1_it_2"] 
f = open(train_file, 'w')
for suffix in suffixes:
    for ff in range(100):
        file_path = f".\\yolov3\\object-detection\\raw_data\\Virtual_FOV_frame_{ff}_{suffix}.jpg"
        if os.path.exists(file_path):
            img_name = f"Virtual_FOV_frame_{ff}_{suffix}"
            f.write(path_template.format(img_name)+'\n')
f.close()
suffixes = ["conf_2_it_0", "conf_2_it_1", "conf_2_it_2"]
f = open(test_file, 'w')
for suffix in suffixes:
    for ff in range(100):
        file_path = f".\\yolov3\\object-detection\\raw_data\\Virtual_FOV_frame_{ff}_{suffix}.jpg"
        if os.path.exists(file_path):
            img_name = f"Virtual_FOV_frame_{ff}_{suffix}"
            f.write(path_template.format(img_name)+'\n')
f.close()

# 2. YOLO Testing

In [ ]:
%matplotlib notebook

In [ ]:
# Test image transformations
ff = 16
suffix = "conf_2_it_1"

In [ ]:
frame = cv2.imread(f'.\\yolov3\\object-detection\\raw_data\\Virtual_FOV_frame_{ff}_{suffix}.jpg')
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
bboxes = np.loadtxt(f'.\\yolov3\\object-detection\\raw_data\\Virtual_FOV_frame_{ff}_{suffix}.txt')

In [ ]:
bboxes

In [ ]:
if len(bboxes.shape) == 1:
    _, xc, yc, w, h = bboxes * np.array([0, 491, 285, 491, 285])
    cv2.rectangle(frame, (int(xc-w/2), int(yc-h/2)), (int(xc+w/2), int(yc+h/2)), (0, 255, 0), 1)
    plt.scatter([xc], [yc], color='black', s=1)
else:
    for bbox in bboxes:
        _, xc, yc, w, h = bbox * np.array([0, 491, 285, 491, 285])
        cv2.rectangle(frame, (int(xc-w/2), int(yc-h/2)), (int(xc+w/2), int(yc+h/2)), (0, 255, 0), 1)
        plt.scatter([xc], [yc], color='black', s=1)
plt.imshow(frame)
#plt.axis('off')
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
%matplotlib notebook

In [ ]:
net = cv2.dnn.readNet('yolov2-voc_last.weights', 'yolov2-voc.cfg')
layer_names = net.getLayerNames()
output_layers = [layer_names[i-1] for i in net.getUnconnectedOutLayers()]
frame = cv2.imread('yolov3/object-detection/raw_data/Virtual_FOV_frame_0_conf_0_it_1.jpg')
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
blob = cv2.dnn.blobFromImage(frame, 1/255, (416, 416), (0, 0, 0), crop=False)

In [ ]:
net.setInput(blob)
outs = net.forward(output_layers)
height, width, channels = frame.shape

In [ ]:
class_ids = []
confidences = []
boxes = []
centers = []
for out in outs:
    for detection in out:
        confidence = detection[5]
        if confidence > 0.7:
            print(confidence)
            print((detection[0]-1/2)*6)
            print((1/2-detection[1])*4)
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))

In [ ]:
for i, bbox in enumerate(boxes):
    x, y, w, h = bbox
    cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 1)
    plt.scatter([x+w/2], [y+h/2], color='black', s=1)
    textbox = plt.text(x, y-3, f"Object: {confidences[i]:0.2f}, ID:{i}", color='black', backgroundcolor='green', fontsize=7)
    textbox.set_bbox(dict(facecolor='green', pad=0.1, edgecolor='green'))
plt.imshow(frame)
#plt.axis('off')
plt.xticks([])
plt.yticks([])
plt.show()